In [ ]:
import os
os.environ['HF_DATASETS_CACHE'] = '/workspace/.cache/huggingface'
os.environ['TRANSFORMERS_CACHE'] = '/workspace/.cache/huggingface'

In [ ]:
import torch
import huggingface_hub
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
huggingface_hub.login('<Your Access Token>')

In [ ]:
model_id = './Llama3-open-Ko-3-8B-Law-Chat-Full-Weights'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto",
)

model.eval()

In [ ]:
question = '''
술을 마시다가 옆테이블에서 먼저 시비를 걸었고 심하게 싸우진 않았습니다
서로 언성이 높아지다가 제가 술김에 화가 나서 화를 참지 못하고 상대의 어깨쪽 팔부분을 꽉 잡았습니다 
당연히 뭐 다치거나 상해입은 부분은 없는데 제가 본인 몸에 손을 댔다는 이유로 폭행죄로 신고를 했고 경찰 조사를 받았습니다 
상대가 합의 의사가 없을 경우 벌금은 어느정도 나올까요??
'''

messages = [
    {"role": "system", "content": "친절한 챗봇으로서 상대방의 요청에 최대한 자세하고 친절하게 답하자. 모든 대답은 한국어(Korean)으로 대답해줘."},
    {"role": "user", "content": question},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

with torch.no_grad():
    outputs = model.generate(
        input_ids,
        max_new_tokens=1024,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.9,
        top_p=0.9,
    )

response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))